In [0]:
!pip install pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

uri = "mongodb+srv://Devendra:<Your Password>@devendra.e3kfvth.mongodb.net/?retryWrites=true&w=majority&appName=Devendra"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

  Using cached pymongo-4.14.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (940 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-23f7628a-e551-44fb-94ca-460a203b0ae9/bin/python -m pip install --upgrade pip' command.


Pinged your deployment. You successfully connected to MongoDB!


In [0]:
db = client.get_database('transaction_predict')
colllection = db['transaction']

In [0]:
data = list(colllection.find())

In [0]:
df = pd.DataFrame(data)
df.head()

,_id,transaction_id,timestamp,user_id,amount,transaction_type,location,location_id,merchant,card_number
0,689f2c23b427ba1ee35cf556,29dc3b24-df7e-4356-a5ea-1ca12b650fa1,1755261637,95469,4896.51,withdrawal,South Williamside,18688,Avila-Nicholson,2237252040848077
1,689f2c25b427ba1ee35cf557,60a5d60d-5c06-4373-afbd-ede6ed59a7f9,1755261642,64331,4708.28,withdrawal,Port Rachel,17758,Henderson PLC,4540256248330661
2,689f2c25b427ba1ee35cf558,e3477f7a-1b43-4e9b-a1c3-4217e7a0bce5,1755261655,71304,4588.16,transfer,North Melissaborough,76748,Hall LLC,4969707302891859
3,689f2c25b427ba1ee35cf559,441beb52-076d-41df-8dc9-1a010525db74,1755261659,88465,1980.23,purchase,Andersontown,27583,"Martin, Joseph and Christensen",30301329953364
4,689f2c25b427ba1ee35cf55a,97c68689-1af5-4bd3-9759-fc0e8f202477,1755261666,74415,3351.97,transfer,Joneschester,57906,Rogers-Peterson,3577239029420040


In [0]:
df.drop(columns = "_id", inplace = True)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
spark = SparkSession.builder.appName("transaction_prediction").getOrCreate()
inventory_df = spark.createDataFrame(df)

In [0]:
inventory_df.show(5)

+--------------------+----------+-------+-------+----------------+--------------------+-----------+--------------------+----------------+
|      transaction_id| timestamp|user_id| amount|transaction_type|            location|location_id|            merchant|     card_number|
+--------------------+----------+-------+-------+----------------+--------------------+-----------+--------------------+----------------+
|29dc3b24-df7e-435...|1755261637|  95469|4896.51|      withdrawal|   South Williamside|      18688|     Avila-Nicholson|2237252040848077|
|60a5d60d-5c06-437...|1755261642|  64331|4708.28|      withdrawal|         Port Rachel|      17758|       Henderson PLC|4540256248330661|
|e3477f7a-1b43-4e9...|1755261655|  71304|4588.16|        transfer|North Melissaborough|      76748|            Hall LLC|4969707302891859|
|441beb52-076d-41d...|1755261659|  88465|1980.23|        purchase|        Andersontown|      27583|Martin, Joseph an...|  30301329953364|
|97c68689-1af5-4bd...|1755261666| 

In [0]:
inventory_df.columns

Out[9]: ['transaction_id',
 'timestamp',
 'user_id',
 'amount',
 'transaction_type',
 'location',
 'location_id',
 'merchant',
 'card_number']

In [0]:
inputColumn = ['amount','location_id']

In [0]:
assembler = VectorAssembler(inputCols=['amount','location_id'], outputCol="features")
data = assembler.transform(inventory_df).select("features", "amount")

In [0]:
data.show()

+-----------------+-------+
|         features| amount|
+-----------------+-------+
|[4896.51,18688.0]|4896.51|
|[4708.28,17758.0]|4708.28|
|[4588.16,76748.0]|4588.16|
|[1980.23,27583.0]|1980.23|
|[3351.97,57906.0]|3351.97|
|[2162.99,68779.0]|2162.99|
| [3277.3,19183.0]| 3277.3|
|[3005.33,71330.0]|3005.33|
|[4163.99,40196.0]|4163.99|
| [125.92,25318.0]| 125.92|
| [894.39,42735.0]| 894.39|
|[3177.23,50958.0]|3177.23|
|[4439.54,89596.0]|4439.54|
|[4407.17,37643.0]|4407.17|
| [969.42,39700.0]| 969.42|
|[2800.61,31516.0]|2800.61|
|[4549.92,59397.0]|4549.92|
|[4953.37,27717.0]|4953.37|
|[4170.56,94997.0]|4170.56|
|[2608.92,55907.0]|2608.92|
+-----------------+-------+
only showing top 20 rows



In [0]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [0]:
rf = RandomForestRegressor(featuresCol="features", labelCol="amount", numTrees= 200)
model = rf.fit(train_data)

In [0]:
predict = model.transform(test_data)

In [0]:
evaluator = RegressionEvaluator(labelCol="amount", predictionCol="prediction", metricName="rmse")
result = evaluator.evaluate(predict)
result

Out[16]: 351.8572833383591

In [0]:
model.save("/Workspace/Users/rahulrock2494@gmail.com/Transaction Prediction")

In [0]:
from pyspark.ml.regression import RandomForestRegressionModel

model = RandomForestRegressionModel.load(
    "/Workspace/Users/rahulrock2494@gmail.com/Transaction Prediction"
)